In [40]:
import pandas as pd
import numpy as np
import copy
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [41]:
fp_active_promos = '../data/active_promos.csv'
fp_executed_promos = '../data/executed_promos.csv'

In [42]:
df_active_promos = pd.read_csv(fp_active_promos, encoding='ISO-8859-1', sep=',')
df_executed_promos = pd.read_csv(fp_executed_promos, encoding='ISO-8859-1', sep=',')

In [43]:
df_active_promos.head(1)

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente
0,297601,29,9,2018-08-06,2018-08-30,3213


In [44]:
# Crear periodo, como el mes de la mediana entre fecha_desde y fecha_hasta
df_active_promos['fecha_desde_dt'] \
  = pd.to_datetime(df_active_promos['Fecha_Desde'], format = '%Y-%m-%d')
df_active_promos['fecha_hasta_dt'] \
  = pd.to_datetime(df_active_promos['Fecha_Hasta'], format = '%Y-%m-%d')

df_active_promos['mid_date'] \
  = df_active_promos['fecha_desde_dt'] + \
    (df_active_promos['fecha_hasta_dt'] - df_active_promos['fecha_desde_dt']) /2

df_active_promos['periodo'] = df_active_promos['mid_date'].apply(lambda dt: dt.replace(day = 1))

df_active_promos['month_year'] = df_active_promos['periodo'].dt.to_period('M')
# Dropear columnas innecesarias
df_active_promos = df_active_promos.drop(['Fecha_Desde', 'Fecha_Hasta', 'fecha_desde_dt',
                                          'fecha_hasta_dt', 'mid_date'], axis = 1)
df_active_promos.head(1)

,CodigoDC,Marca,Cupo,Cliente,periodo,month_year
0,297601,29,9,3213,2018-08-01,2018-08


In [45]:
# Crear un lienzo para todas las Combinaciones 
l_unique_periodo = list(df_active_promos['month_year'].unique())
l_unique_cliente = list(df_active_promos['Cliente'].unique())

l_unique_periodo_fin = l_unique_periodo * len(l_unique_cliente)
l_unique_cliente_fin = [[cliente]*len(l_unique_periodo) for cliente in l_unique_cliente]
#flatten
l_unique_cliente_fin = [item for sublist in l_unique_cliente_fin for item in sublist]

# Creamos un dataframe lienzo
df_active_promos_base = pd.DataFrame(
  list(zip(l_unique_periodo_fin, l_unique_cliente_fin)),
  columns = ['month_year', 'Cliente']
)

# Dropear mes sin data
df_active_promos_base = df_active_promos_base[df_active_promos_base['month_year']!=pd.Period('2019-09', 'M')]

df_active_promos_base.head(1)

,month_year,Cliente
0,2018-08,3213


In [46]:
print(f"Longitud de lienzo antes: {len(df_active_promos_base)}")
df_active_promos_base = pd.merge(
  df_active_promos_base,
  df_active_promos,
  how = 'left',
  on = ['Cliente', 'month_year']
)
print(f"Longitud de lienzo antes: {len(df_active_promos_base)}")

Longitud de lienzo antes: 176332
Longitud de lienzo antes: 483993


In [47]:
df_active_promos_base.isna().sum()

month_year        0
Cliente           0
CodigoDC      98191
Marca         98191
Cupo          98191
periodo       98191
dtype: int64

In [48]:
df_active_promos_base[df_active_promos_base['CodigoDC'].isna()]

,month_year,Cliente,CodigoDC,Marca,Cupo,periodo
26,2018-12,3213,NaN,NaN,NaN,NaT
33,2019-02,3213,NaN,NaN,NaN,NaT
38,2019-04,3213,NaN,NaN,NaN,NaT
43,2019-06,3213,NaN,NaN,NaN,NaT
84,2018-12,3795,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...
483987,2019-03,11167,NaN,NaN,NaN,NaT
483988,2019-04,11167,NaN,NaN,NaN,NaT
483989,2019-07,11167,NaN,NaN,NaN,NaT
483990,2019-05,11167,NaN,NaN,NaN,NaT


In [49]:
a = list(df_active_promos_base['month_year'].unique())
a.sort()
a

[Period('2018-08', 'M'),
 Period('2018-09', 'M'),
 Period('2018-10', 'M'),
 Period('2018-11', 'M'),
 Period('2018-12', 'M'),
 Period('2019-01', 'M'),
 Period('2019-02', 'M'),
 Period('2019-03', 'M'),
 Period('2019-04', 'M'),
 Period('2019-05', 'M'),
 Period('2019-06', 'M'),
 Period('2019-07', 'M'),
 Period('2019-08', 'M')]

In [50]:
df_ap = copy.deepcopy(df_active_promos_base)
df_ap = df_ap.drop('periodo', axis=1)
df_ap.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo
0,2018-08,3213,297601.0,29.0,9.0


In [51]:
# Creamos agrupado por cliente_month year
df_g1 = df_ap.groupby(['month_year', 'Cliente']).agg({
  'CodigoDC': 'count',
}).reset_index()

df_g1 = df_g1.rename(columns = {'CodigoDC': 'num_ofertas'})

df_cliente1 = df_g1[df_g1['Cliente']==1]
df_cliente1.sort_values('month_year').head(20)

,month_year,Cliente,num_ofertas
0,2018-08,1,0
13564,2018-09,1,8
27128,2018-10,1,6
40692,2018-11,1,8
54256,2018-12,1,0
67820,2019-01,1,0
81384,2019-02,1,0
94948,2019-03,1,4
108512,2019-04,1,0
122076,2019-05,1,0


In [52]:
# Calcular acumulados hacia el pasado
test_dict = {
  'num_ofertas': [2],
}

In [53]:
df_g2 = copy.deepcopy(df_g1)
for col_name, months_past_list in test_dict.items():
  for window in months_past_list:
    #suma
    rol_col_name = 'sum' + '_' + str(window) + col_name
    #quitamos periodo de lista grupo    
    df_g2[rol_col_name] = df_g2.sort_values('month_year', ascending=True)\
                            .groupby('Cliente')\
                            ['num_ofertas'].apply(lambda x: x.rolling(window).sum())
    #llenar nan de estos agrupados con la media del grupo
    df_g2[rol_col_name] = df_g2.groupby('Cliente')\
                            [rol_col_name].transform(lambda x: x.fillna(x.mean()))    
    # media
    rol_col_name = 'mean' + '_' + str(window) + col_name
    df_g2[rol_col_name] = df_g2.sort_values('month_year', ascending=True)\
                            .groupby('Cliente')\
                           ['num_ofertas'].apply(lambda x: x.rolling(window).mean())
    #llenar nan de estos agrupados con la media del grupo
    df_g2[rol_col_name] = df_g2.groupby('Cliente')\
                            [rol_col_name].transform(lambda x: x.fillna(x.mean()))     

In [54]:
df_g2.head(5)

,month_year,Cliente,num_ofertas,sum_2num_ofertas,mean_2num_ofertas
0,2018-08,1,0,5.083333,2.541667
1,2018-08,2,0,5.500000,2.750000
2,2018-08,4,0,0.250000,0.125000
3,2018-08,6,0,5.416667,2.708333
4,2018-08,7,0,4.666667,2.333333


In [55]:
# 
import pickle
fp_output = '../tablones/data_grupo2_ofertas.pkl'
def store_pickle(fp, obj):
  with open(fp, 'wb') as f:
    pickle.dump(obj, f)
    
store_pickle(fp_output, df_g2)

## Analisis para ejecutadas, ofertadas y proporcion

In [56]:
# lo mismo pero para ejecutadas
df_executed_promos['label'] = 1
df_executed_promos.head(1)

,CodigoDC,Cliente,Marca,Cupo,label
0,297601,8410,29,9,1


In [57]:
df_ap.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo
0,2018-08,3213,297601.0,29.0,9.0


In [58]:
df_ap.isna().sum()

month_year        0
Cliente           0
CodigoDC      98191
Marca         98191
Cupo          98191
dtype: int64

In [59]:
df_g1 = pd.merge(
  df_ap,
  df_executed_promos,
  how = 'left',
  on = ['Cliente', 'Marca', 'Cupo', 'CodigoDC']
)
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,NaN


In [60]:
df_g1['label'].value_counts()

1.0    27791
Name: label, dtype: int64

In [61]:
df_g1.isna().sum()

month_year         0
Cliente            0
CodigoDC       98191
Marca          98191
Cupo           98191
label         456202
dtype: int64

In [62]:
# Cuidado
# LLenar label con 0 cuando Marca NO ES NAN

In [63]:
#df_g1['temp_na'] = df_g1[]
missing_mask = (~df_g1['Marca'].isna()) & (df_g1['label'] != 1.0)
df_g1.loc[missing_mask, 'label'] = 0

In [64]:
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,0.0


In [65]:
df_g1.isna().sum()

month_year        0
Cliente           0
CodigoDC      98191
Marca         98191
Cupo          98191
label         98191
dtype: int64

In [66]:
df_g1['label'].value_counts()

0.0    358011
1.0     27791
Name: label, dtype: int64

In [67]:
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,0.0


In [68]:
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,0.0


In [69]:
# Creamos agrupado por cliente_month year
df_g1 = df_g1.groupby(['month_year', 'Cliente']).agg({
  'label': ['count', 'sum'],
}).reset_index()

df_g1.columns = ['_'.join(col).rstrip('_') for col in df_g1.columns.values]
df_g1.head(1)
# df_g1 = df_g1.rename(columns = {'label': 'num_prom_tomadas'})

,month_year,Cliente,label_count,label_sum
0,2018-08,1,0,0.0


In [70]:
# Renombrar columnas
df_g1 = df_g1.rename(columns={'label_count': 'ofertas_dadas', 'label_sum': 'ofertas_tomadas'})
df_g1['proporcion_tomadas'] = df_g1['ofertas_tomadas'] / (df_g1['ofertas_dadas']+0.0000001)
df_g1.head(1)

,month_year,Cliente,ofertas_dadas,ofertas_tomadas,proporcion_tomadas
0,2018-08,1,0,0.0,0.0


In [71]:
df_g1.isna().sum()

month_year            0
Cliente               0
ofertas_dadas         0
ofertas_tomadas       0
proporcion_tomadas    0
dtype: int64

In [72]:
# Crear acumulados en el pasado para ofertas_dadas, ofertas_tomadas, proporcion_tomadas
# Agrupando por Cliente

In [73]:
# Calcular acumulados hacia el pasado
test_dict = {
  'ofertas_dadas': [2,],
  'ofertas_tomadas': [2,],
  'proporcion_tomadas': [2,],
}

In [74]:
df_g2 = copy.deepcopy(df_g1)
for col_name, months_past_list in test_dict.items():
  for window in months_past_list:
    #suma
    rol_col_name = 'sum' + '_' + str(window) + col_name
    #quitamos periodo de lista grupo    
    df_g2[rol_col_name] = df_g2.sort_values('month_year', ascending=True)\
                            .groupby('Cliente')\
                            [col_name].apply(lambda x: x.rolling(window).sum())
    #llenar nan de estos agrupados con la media del grupo
    df_g2[rol_col_name] = df_g2.groupby('Cliente')\
                            [rol_col_name].transform(lambda x: x.fillna(x.mean()))    
    # media
    rol_col_name = 'mean' + '_' + str(window) + col_name
    df_g2[rol_col_name] = df_g2.sort_values('month_year', ascending=True)\
                            .groupby('Cliente')\
                           [col_name].apply(lambda x: x.rolling(window).mean())
    #llenar nan de estos agrupados con la media del grupo
    df_g2[rol_col_name] = df_g2.groupby('Cliente')\
                            [rol_col_name].transform(lambda x: x.fillna(x.mean()))     

In [75]:
df_g2.head(1)

,month_year,Cliente,ofertas_dadas,ofertas_tomadas,proporcion_tomadas,sum_2ofertas_dadas,mean_2ofertas_dadas,sum_2ofertas_tomadas,mean_2ofertas_tomadas,sum_2proporcion_tomadas,mean_2proporcion_tomadas
0,2018-08,1,0,0.0,0.0,5.083333,2.541667,0.166667,0.083333,0.020833,0.010417


In [76]:
df_g2[df_g2['Cliente']==1]

,month_year,Cliente,ofertas_dadas,ofertas_tomadas,proporcion_tomadas,sum_2ofertas_dadas,mean_2ofertas_dadas,sum_2ofertas_tomadas,mean_2ofertas_tomadas,sum_2proporcion_tomadas,mean_2proporcion_tomadas
0,2018-08,1,0,0.0,0.000,5.083333,2.541667,0.166667,0.083333,0.020833,0.010417
13564,2018-09,1,8,0.0,0.000,8.000000,4.000000,0.000000,0.000000,0.000000,0.000000
27128,2018-10,1,6,0.0,0.000,14.000000,7.000000,0.000000,0.000000,0.000000,0.000000
40692,2018-11,1,8,1.0,0.125,14.000000,7.000000,1.000000,0.500000,0.125000,0.062500
54256,2018-12,1,0,0.0,0.000,8.000000,4.000000,1.000000,0.500000,0.125000,0.062500
67820,2019-01,1,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
81384,2019-02,1,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
94948,2019-03,1,4,0.0,0.000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000
108512,2019-04,1,0,0.0,0.000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000
122076,2019-05,1,0,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [77]:
# ESTO LO AGREGUE YO PARA PROBAR
df_g2 = df_g2[['month_year', 'Cliente', 'ofertas_dadas',]]

In [78]:
# Almacenar df_g2
fp_output = '../tablones/data_grupo2_proporcion.pkl'    
store_pickle(fp_output, df_g2)

## Analisis por el grupo (month_year, Cliente, Marca, Cupo)

In [246]:
# lo mismo pero para month_year, Cliente, marca, cupo

In [247]:
# lo mismo pero para ejecutadas
df_executed_promos['label'] = 1
df_executed_promos.head(1)

,CodigoDC,Cliente,Marca,Cupo,label
0,297601,8410,29,9,1


In [248]:
df_ap.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo
0,2018-08,3213,297601.0,29.0,9.0


In [249]:
df_ap.isna().sum()

month_year        0
Cliente           0
CodigoDC      98191
Marca         98191
Cupo          98191
dtype: int64

In [250]:
df_g1 = pd.merge(
  df_ap,
  df_executed_promos,
  how = 'left',
  on = ['Cliente', 'Marca', 'Cupo', 'CodigoDC']
)
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,NaN


In [251]:
df_g1['label'].value_counts()

1.0    27791
Name: label, dtype: int64

In [252]:
df_g1.isna().sum()

month_year         0
Cliente            0
CodigoDC       98191
Marca          98191
Cupo           98191
label         456202
dtype: int64

In [253]:
# Cuidado
# LLenar label con 0 cuando Marca NO ES NAN

In [254]:
#df_g1['temp_na'] = df_g1[]
missing_mask = (~df_g1['Marca'].isna()) & (df_g1['label'] != 1.0)
df_g1.loc[missing_mask, 'label'] = 0

In [255]:
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,0.0


In [256]:
df_g1.isna().sum()

month_year        0
Cliente           0
CodigoDC      98191
Marca         98191
Cupo          98191
label         98191
dtype: int64

In [257]:
df_g1['label'].value_counts()

0.0    358011
1.0     27791
Name: label, dtype: int64

In [258]:
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,0.0


In [259]:
df_g1.head(1)

,month_year,Cliente,CodigoDC,Marca,Cupo,label
0,2018-08,3213,297601.0,29.0,9.0,0.0


In [260]:
# Creamos agrupado por cliente_month year
df_g1 = df_g1.groupby(['month_year', 'Cliente', 'Marca', 'Cupo']).agg({
  'label': ['count', 'sum'],
}).reset_index()

df_g1.columns = ['_'.join(col).rstrip('_') for col in df_g1.columns.values]
df_g1.head(1)
# df_g1 = df_g1.rename(columns = {'label': 'num_prom_tomadas'})

,month_year,Cliente,Marca,Cupo,label_count,label_sum
0,2018-08,140,40.0,16.0,1,1.0


In [261]:
# Renombrar columnas
df_g1 = df_g1.rename(columns={
  'label_count': 'ofertas_dadas_mc',
  'label_sum': 'ofertas_tomadas_mc'
})
df_g1['proporcion_tomadas_mc'] = df_g1['ofertas_tomadas_mc'] / (df_g1['ofertas_dadas_mc']+0.0000001)
df_g1.head(1)

,month_year,Cliente,Marca,Cupo,ofertas_dadas_mc,ofertas_tomadas_mc,proporcion_tomadas_mc
0,2018-08,140,40.0,16.0,1,1.0,1.0


In [262]:
df_g1.isna().sum()

month_year               0
Cliente                  0
Marca                    0
Cupo                     0
ofertas_dadas_mc         0
ofertas_tomadas_mc       0
proporcion_tomadas_mc    0
dtype: int64

In [263]:
# Crear acumulados en el pasado para ofertas_dadas, ofertas_tomadas, proporcion_tomadas
# Agrupando por Cliente

In [264]:
# Calcular acumulados hacia el pasado
test_dict = {
  'ofertas_dadas_mc': [2,3,4,5,6],
  'ofertas_tomadas_mc': [2,3,4,5,6],
  'proporcion_tomadas_mc': [2,3,4,5,6],
}

In [265]:
df_g2 = copy.deepcopy(df_g1)
for col_name, months_past_list in test_dict.items():
  for window in months_past_list:
    #suma
    rol_col_name = 'sum' + '_' + str(window) + col_name
    #quitamos periodo de lista grupo    
    df_g2[rol_col_name] = df_g2.sort_values('month_year', ascending=True)\
                            .groupby(['Cliente', 'Marca', 'Cupo'])\
                            [col_name].apply(lambda x: x.rolling(window).sum())
    #llenar nan de estos agrupados con la media del grupo
    df_g2[rol_col_name] = df_g2.groupby(['Cliente', 'Marca', 'Cupo'])\
                            [rol_col_name].transform(lambda x: x.fillna(x.mean()))    
    # media
    rol_col_name = 'mean' + '_' + str(window) + col_name
    df_g2[rol_col_name] = df_g2.sort_values('month_year', ascending=True)\
                            .groupby(['Cliente', 'Marca', 'Cupo'])\
                           [col_name].apply(lambda x: x.rolling(window).mean())
    #llenar nan de estos agrupados con la media del grupo
    df_g2[rol_col_name] = df_g2.groupby(['Cliente', 'Marca', 'Cupo'])\
                            [rol_col_name].transform(lambda x: x.fillna(x.mean()))     

In [266]:
df_g2.head(1)

,month_year,Cliente,Marca,Cupo,ofertas_dadas_mc,ofertas_tomadas_mc,proporcion_tomadas_mc,sum_2ofertas_dadas_mc,mean_2ofertas_dadas_mc,sum_3ofertas_dadas_mc,mean_3ofertas_dadas_mc,sum_4ofertas_dadas_mc,mean_4ofertas_dadas_mc,sum_5ofertas_dadas_mc,mean_5ofertas_dadas_mc,sum_6ofertas_dadas_mc,mean_6ofertas_dadas_mc,sum_2ofertas_tomadas_mc,mean_2ofertas_tomadas_mc,sum_3ofertas_tomadas_mc,mean_3ofertas_tomadas_mc,sum_4ofertas_tomadas_mc,mean_4ofertas_tomadas_mc,sum_5ofertas_tomadas_mc,mean_5ofertas_tomadas_mc,sum_6ofertas_tomadas_mc,mean_6ofertas_tomadas_mc,sum_2proporcion_tomadas_mc,mean_2proporcion_tomadas_mc,sum_3proporcion_tomadas_mc,mean_3proporcion_tomadas_mc,sum_4proporcion_tomadas_mc,mean_4proporcion_tomadas_mc,sum_5proporcion_tomadas_mc,mean_5proporcion_tomadas_mc,sum_6proporcion_tomadas_mc,mean_6proporcion_tomadas_mc
0,2018-08,140,40.0,16.0,1,1.0,1.0,4.5,2.25,6.714286,2.238095,9.0,2.25,11.0,2.2,13.5,2.25,1.5,0.75,2.0,0.666667,2.5,0.625,3.0,0.6,3.75,0.625,0.695833,0.347917,0.871429,0.290476,1.05,0.2625,1.26,0.252,1.575,0.2625


In [269]:
df_g2[df_g2['sum_2ofertas_dadas_mc'].isna()]

,month_year,Cliente,Marca,Cupo,ofertas_dadas_mc,ofertas_tomadas_mc,proporcion_tomadas_mc,sum_2ofertas_dadas_mc,mean_2ofertas_dadas_mc,sum_3ofertas_dadas_mc,mean_3ofertas_dadas_mc,sum_4ofertas_dadas_mc,mean_4ofertas_dadas_mc,sum_5ofertas_dadas_mc,mean_5ofertas_dadas_mc,sum_6ofertas_dadas_mc,mean_6ofertas_dadas_mc,sum_2ofertas_tomadas_mc,mean_2ofertas_tomadas_mc,sum_3ofertas_tomadas_mc,mean_3ofertas_tomadas_mc,sum_4ofertas_tomadas_mc,mean_4ofertas_tomadas_mc,sum_5ofertas_tomadas_mc,mean_5ofertas_tomadas_mc,sum_6ofertas_tomadas_mc,mean_6ofertas_tomadas_mc,sum_2proporcion_tomadas_mc,mean_2proporcion_tomadas_mc,sum_3proporcion_tomadas_mc,mean_3proporcion_tomadas_mc,sum_4proporcion_tomadas_mc,mean_4proporcion_tomadas_mc,sum_5proporcion_tomadas_mc,mean_5proporcion_tomadas_mc,sum_6proporcion_tomadas_mc,mean_6proporcion_tomadas_mc
967,2018-08,5937,40.0,16.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1016,2018-08,6187,40.0,16.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1054,2018-08,6394,40.0,16.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1303,2018-08,8037,40.0,16.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1311,2018-08,8066,40.0,16.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151350,2019-08,15059,40.0,16.0,2,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151358,2019-08,15065,29.0,9.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151359,2019-08,15066,29.0,9.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151360,2019-08,15067,29.0,9.0,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [199]:
# Almacenar df_g2
# fp_output = '../tablones/data_grupo2_proporcion_MC.pkl'    
# store_pickle(fp_output, df_g2)